In [12]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta

# Define the date range
start_date = datetime.strptime('2023-04-25', '%Y-%m-%d')
end_date = datetime.strptime('2024-04-24', '%Y-%m-%d')

# Define the count of records to generate
count = 1000

# Create a Faker instance
faker = Faker()

# Generate dummy data
data = []
for c in range(count):
    date = faker.date_time_between_dates(datetime_start=start_date, datetime_end=end_date).strftime('%Y-%m-%d')
    original_image_url = faker.image_url(width=640, height=480, placeholder_url=f"https://original_input_image_{c}.jpg")
    model_output_url = faker.image_url(width=640, height=480, placeholder_url=f"https://model_output_image_{c}.png")
    manual_clean_url = faker.image_url(width=640, height=480, placeholder_url=f"https://manually_cleaned_image_{c}.png") if random.choice([True, False]) else None
    data.append({'date': date, 'original_image_url': original_image_url, 'model_output_url': model_output_url, 'manual_clean_url': manual_clean_url})

# Create a DataFrame
df = pd.DataFrame(data)


In [13]:
df.head()

,date,original_image_url,model_output_url,manual_clean_url
0,2023-07-01,https://original_input_image_0.jpg,https://model_output_image_0.png,https://manually_cleaned_image_0.png
1,2023-11-25,https://original_input_image_1.jpg,https://model_output_image_1.png,None
2,2023-09-08,https://original_input_image_2.jpg,https://model_output_image_2.png,https://manually_cleaned_image_2.png
3,2023-12-19,https://original_input_image_3.jpg,https://model_output_image_3.png,https://manually_cleaned_image_3.png
4,2023-08-05,https://original_input_image_4.jpg,https://model_output_image_4.png,None


In [14]:
df = df.sort_values('date', ascending=False).reset_index(drop=True)
# Create a new column 'manually_cleaned' that is 1 if 'manual_clean_url' is not empty
df['manually_cleaned'] = df['manual_clean_url'].apply(lambda x: 0 if pd.isna(x) else 1)

In [15]:
df.head()

,date,original_image_url,model_output_url,manual_clean_url,manually_cleaned
0,2024-04-23,https://original_input_image_24.jpg,https://model_output_image_24.png,https://manually_cleaned_image_24.png,1
1,2024-04-23,https://original_input_image_863.jpg,https://model_output_image_863.png,https://manually_cleaned_image_863.png,1
2,2024-04-22,https://original_input_image_870.jpg,https://model_output_image_870.png,None,0
3,2024-04-22,https://original_input_image_762.jpg,https://model_output_image_762.png,https://manually_cleaned_image_762.png,1
4,2024-04-22,https://original_input_image_335.jpg,https://model_output_image_335.png,None,0


In [16]:
len(df[df['manually_cleaned']==1])/len(df)

0.503

In [17]:
df.to_csv('daily_report.csv',index=False)

In [19]:
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd

app = dash.Dash(__name__)

# Load the CSV file
df = pd.read_csv('daily_report.csv', parse_dates=['date'])

# Create a new column 'manually_cleaned' that is 1 if 'manual_clean_url' is not empty
df['manually_cleaned'] = df['manual_clean_url'].apply(lambda x: 0 if pd.isna(x) else 1)

app.layout = html.Div([
    html.H1("Image Processing Dashboard"),
    dcc.Graph(id='image-stats'),
    dcc.DatePickerRange(
        id='date-range',
        min_date_allowed=df['date'].min(),
        max_date_allowed=df['date'].max(),
        initial_visible_month=df['date'].max(),
        start_date=df['date'].min(),
        end_date=df['date'].max()
    )
])

@app.callback(
    Output('image-stats', 'figure'),
    [Input('date-range', 'start_date'),
     Input('date-range', 'end_date')]
)
def update_figure(start_date, end_date):
    filtered_df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

    # Total images per day
    total_images = filtered_df.groupby(filtered_df['date'].dt.date).size()
    # Total manually cleaned images per day
    manually_cleaned_images = filtered_df.groupby(filtered_df['date'].dt.date)['manually_cleaned'].sum()

    # Create traces
    trace1 = go.Scatter(
        x=total_images.index,
        y=total_images.values,
        mode='lines+markers',
        name='Total Images'
    )
    trace2 = go.Scatter(
        x=manually_cleaned_images.index,
        y=manually_cleaned_images.values,
        mode='lines+markers',
        name='Manually Cleaned Images'
    )

    # Bar chart trace
    trace3 = go.Bar(
        x=total_images.index,
        y=total_images.values,
        name='Total Images (Bar)',
        hoverinfo='text',
        text=['Manual Cleaning %: {:.2f}%'.format((mc / ti) * 100) if ti != 0 else 'Manual Cleaning %: 0%' for ti, mc in zip(total_images.values, manually_cleaned_images.values)],
        textposition='inside',
        opacity=0.5
    )

    return {
        'data': [trace1, trace2, trace3],
        'layout': go.Layout(
            title='Daily Image Processing Statistics',
            xaxis={'title': 'Date'},
            yaxis={'title': 'Number of Images'},
            margin={'l': 40, 'b': 40, 't': 50, 'r': 10},
            hovermode='closest'
        )
    }

if __name__ == '__main__':
    app.run_server(debug=True, host='0.0.0.0', port=8051)
